In [ ]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# Sky visualization
from ipyaladin import Aladin  # version >=0.4.0

# For plots
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from astroquery.simbad import Simbad
from astropy.cosmology import FlatLambdaCDM
import astropy.units as uu
import astropy.constants as cc

# Define the cosmology
cosmo = FlatLambdaCDM(H0=67.8 * uu.km / uu.s / uu.Mpc, Om0=0.308)


# the catalogue name in VizieR
CATALOGUE = "J/MNRAS/390/466"
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

tables = voresource.get_tables()
print(f"In this catalogue, we have {len(tables)} tables.")
for table_name, table in tables.items():
    print(f"{table_name}: {table.description}")
tables_names = list(tables.keys())


# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select * from "{first_table_name}"',
)
tap_records

In [ ]:
df = tap_records.to_table().to_pandas()
df.columns

In [ ]:
def calcBMAG(z, flux, distance):
    # Convert flux to apparent magnitude (assuming flux is in Janskys and using B-band as example)
    apparent_mag = -2.5 * np.log10(flux / 3631)
    
    # Calculate distance modulus
    distance_modulus = 5 * np.log10(distance.to(uu.pc).value / 10)
    
    # Calculate absolute magnitude
    b_mag = apparent_mag - distance_modulus
    # print(f"BMag: {b_mag}")
    return b_mag

def hms_to_deg(ra_hms):
    hours, minutes, seconds = map(float, ra_hms.split())
    return (hours + minutes/60 + seconds/3600) * 15
    
def helio_to_cmb(z_helio, ra, dec):
    # Constants
    c = 299792.458  # speed of light in km/s
    v_sun_cmb = 370  # Sun's velocity relative to CMB in km/s
    lambda_cmb = 168.0  # RA in degrees
    beta_cmb = -7.0  # Dec in degrees
    
    # Convert angles from degrees to radians
    lambda_cmb_rad = np.radians(lambda_cmb)
    beta_cmb_rad = np.radians(beta_cmb)
    ra_rad = np.radians(ra)
    dec_rad = np.radians(dec)
    
    # Calculate the projection of Sun's motion on the line of sight to the object
    cos_theta = (np.sin(beta_cmb_rad) * np.sin(dec_rad) +
                 np.cos(beta_cmb_rad) * np.cos(dec_rad) * np.cos(ra_rad - lambda_cmb_rad))
    v_los = v_sun_cmb * cos_theta  # line of sight component of the Sun's motion

    # Convert heliocentric redshift to CMB redshift
    z_cmb = (z_helio * c + v_los) / c

    return z_cmb

# Add the 'z_value' field to the query to get the redshift
Simbad.add_votable_fields('z_value')
Simbad.add_votable_fields('flux(B)')  # Adds B-band magnitudes
df["z_helio"] = np.nan
df["z_cmb"] = np.nan
df["BMAG_Simbad"] = np.nan
df["Dist_Helio_z"] = np.nan
df['Dist_CMB_z'] = np.nan
df['Vmax'] = df['Vmax'].astype('float64')
R0 = (14.01E9*uu.lyr).to(uu.Mpc).value

# Iterate over the DataFrame
for index, row in df.iterrows():
    obj = Simbad.query_object(row['Name'])
    if obj is not None and len(obj) > 0:
        df.at[index, 'z_helio'] = obj['Z_VALUE'][0] if obj['Z_VALUE'][0] else np.nan
        df.at[index, 'BMAG_Simbad'] = obj['FLUX_B'][0] if 'FLUX_B' in obj.columns and not obj['FLUX_B'].mask[0] else np.nan
        # Calculate z_cmb using the function defined above
        if not np.isnan(df.at[index, 'z_helio']):
            z_cmb = df.at[index, 'z_cmb'] = helio_to_cmb(df.at[index, 'z_helio'], row['RAJ2000'], row['DEJ2000'])
            df.at[index, 'Dist_Helio_z'] = cosmo.luminosity_distance(df.at[index, 'z_helio']).to(uu.Mpc).value
            df.at[index, 'Dist_CMB_z'] = R0*z_cmb/(1+z_cmb)
# Constants
zero_point = 48.6  # Adjust based on calibration            
df["BMAG_calc_helio_z_Flux"]= [ calcBMAG(df.z_helio, x,y*uu.Mpc)  for x, y in zip(df.Flux, df.Dist)]
df['apparent_mag'] = -2.5 * np.log10(df['Flux']) + zero_point
df['BMAG_CMB'] = df['apparent_mag'] - 5 * np.log10(df['Dist_CMB_z']/10)
# Remove rows where 'Vmax' is NaN
df = df.dropna(subset=['Vmax', "BMAG", "BMAG_CMB"])
df = df[df['Vmax'] > 0]
df = df[df['Flux'] > 0]

# Replace negative 'Vmax' values
df.loc[df['Vmax'] < 0, 'Vmax'] = 1E-4
df.to_excel("./Ghasp_Data/df_GHasp_Simbad.xlsx")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Constants
zero_point = 48.6  # Adjust based on calibration

X = np.log10(df['Vmax'].values.reshape(-1, 1))
# Calculate apparent magnitude from Flux
Y = df['BMAG_CMB'].values.reshape(-1, 1)

# Ensure that 'X' and 'Y' are not containing NaN values
if np.isnan(X).any() or np.isnan(Y).any():
    print("Warning: NaN values detected in X or Y, check your data preprocessing steps.")

# Linear regression model
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # Perform linear regression
Y_pred = linear_regressor.predict(X)  # Make predictions

# Output the coefficients
alpha = linear_regressor.coef_[0][0]
beta = linear_regressor.intercept_[0]

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(X, Y, c='blue', label='Data Points')
plt.plot(X, Y_pred, color='red', label='Fit: Y={:.2f}X + {:.2f}'.format(alpha, beta))
plt.xlabel('LogVmax (km/s)')
plt.ylabel('BMAG_CMB')
plt.title('Tully-Fisher Relation')
plt.gca().invert_yaxis()  # Invert y-axis as brighter objects have lower magnitude
plt.legend()
plt.grid(True)
plt.show()

print(f"Coefficient (alpha): {alpha}, Intercept (beta): {beta}")
